In [ ]:
#attempt to improve the NN
#add the local and the global view construct coz transit false positive mismatch seems to be a major problem
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
temp1=np.loadtxt('training_data/Xtrain_av_raw.csv',delimiter=',')
temp2=np.loadtxt('training_data/Xtrain_av_raw_v2.csv',delimiter=',')
temp3=np.loadtxt('training_data/Xtrain_av_raw_v3.csv',delimiter=',')
temp4=np.loadtxt('training_data/Xtrain_av_raw_v4.csv',delimiter=',')
print(temp1.shape)
print(temp2.shape)
print(temp3.shape)
print(temp4.shape)


In [ ]:
acc_func=[]
val_acc_func=[]
test_accuracy=[]

In [ ]:
#main code:
#Q1. ARE MORE LAYERS BETTER? We use 2 classifier Training sample to test this
def are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,img_size,no_l,epoch):
    #add hidden layers
    conv_ip = keras.layers.Input(shape=(img_size,),name='Input')
    x=keras.layers.Reshape((img_size, 1), input_shape=(img_size,),name='reshape_1')(conv_ip)
    x=keras.layers.BatchNormalization()(x)

    for i in range(0,no_l):
        x=keras.layers.Conv1D(16,kernel_size=3,strides=1,activation='tanh',name='conv16_3_'+str(i))(x)
    x=keras.layers.MaxPool1D(3,strides=2 ,data_format='channels_last',name='maxpool_1')(x)
    for i in range(0,no_l):
        x=keras.layers.Conv1D(32,kernel_size=3,strides=1,activation='tanh',name='conv32_3_'+str(i))(x)
    x=keras.layers.MaxPool1D(3,strides=2,data_format='channels_last',name='maxpool_2')(x)
    x=keras.layers.Flatten(name='flat_1')(x)
    x3=keras.layers.Dense(128,name='dense_layer_1',activation='relu')(x)
    x3=keras.layers.Dense(128,name='dense_layer_3',activation='relu')(x3)
    conv_op=keras.layers.Dense(2,name='dense_layer_4',activation='softmax')(x3)

    convNN = keras.Model(inputs=conv_ip, outputs=conv_op,name='Convolutional_NN')
    convNN.summary()
    convNN.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00005), loss='categorical_crossentropy',metrics=['accuracy'])
    es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    history=convNN.fit(np.array(Xtrain),np.array(Ytrain), batch_size=64, epochs=epoch , verbose=1 , validation_split=0.2,callbacks=[es_callback])

    fig,ax=plt.subplots(1,1,figsize=(5,5))
    ax.plot(history.history['accuracy'])
    ax.plot(history.history['val_accuracy'])
    ax.set_title('model Accuracy')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('epoch')
    ax.legend(['train', 'test'], loc='upper left')
    #plt.savefig('loop_acc_100_3d0'+str(no_l)+'_'+str(img_size)+'.png')

    test_loss, test_acc = convNN.evaluate(np.array(Xtest), np.array(Ytest))
    print('Test accuracy:', test_acc)
    test_accuracy.append(test_acc)
    acc_func.append(history.history['accuracy'])
    val_acc_func.append(history.history['val_accuracy'])
    #plt.show()



In [ ]:
X_train=np.loadtxt('training_data/Xtrain_av_raw200_2_2d5_v2.csv',delimiter=',')
Y_train=np.loadtxt('training_data/Ytrain_av_raw200_2_2d5_v2.csv',delimiter=',')
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_train, Y_train, test_size=0.1)

print("shapes:",Xtrain.shape,Ytrain.shape,Xtest.shape,Ytest.shape)



In [ ]:

are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,200,1,60)
are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,200,2,60)
are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,200,3,60)
are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,200,4,60)
are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,200,5,60)
are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,200,6,60)
are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,200,7,60)
are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,200,8,60)
are_more_layers_better(Xtrain,Ytrain,Xtest,Ytest,200,9,60)


In [ ]:
#print(test_accuracy)
#print(acc_func)
#print(val_acc_func)
new_mean=[el[-10:] for el in val_acc_func]
print(new_mean)
new_mean=[np.mean(el[-10:]) for el in val_acc_func]
fig2,ax2=plt.subplots(1,1,figsize=(5,5))
ax2.plot(np.arange(1,len(test_accuracy)+1,1),new_mean)
ax2.set_xlabel('number of layers')
ax2.set_ylabel('test accuracy')
plt.savefig('test_acc_progression_2d5_200.png')

fig3,ax3=plt.subplots(1,1,figsize=(7,7))
for i in range(0,len(val_acc_func)):
    ax3.plot(acc_func[i],label="train_lay"+str(i+1))
    ax3.plot(val_acc_func[i],label="test_lay"+str(i+1))
ax3.legend()
ax3.set_xlabel('epoch') 
ax3.set_ylabel('accuracy')

#np.savetxt('comprehensive_cnn_acc.csv',acc_func,delimiter=',')
plt.savefig('lc_cumulative_2d5_200.png')
plt.show()